In [ ]:
import os
import torch
import rasterio
import geopandas as gpd
import numpy as np
import json
import fiona
from samgeo.hq_sam import SamGeo
import GeoUtils
import GeoFeatures
import SegmentPlots
import GeoRegistration
from shapely.geometry import Point
import math
import matplotlib.pyplot as plt

## Initialize variables

In [ ]:
sam_model_path = os.path.join('sam_hq_vit_h.pth')
downSampleFactor = 2
crs = "EPSG:4326"
folderName = 'Data'
s1_FileName = '03-02-2023'
s1_FolderPath = os.path.join(folderName, s1_FileName)
s1_img_input = os.path.join(s1_FolderPath, s1_FileName + '.tif')

s2_FileName = '13-03-2023'
s2_FolderPath = os.path.join(folderName, s2_FileName)
s2_img_input = os.path.join(s2_FolderPath, s2_FileName + '.tif')

gcp_template_file_path = os.path.join(folderName, 'Results', 'points.txt')
gcp_out_file = os.path.join(folderName, 'Results', s1_FileName+'-'+s2_FileName+'.points')

bins_dist = 24
bins_brg = 24
min_Neighbors = 5 
neighbor_percentile = 90 

In [ ]:
# Downsample the orthomosaics for reducing subsequent computational complexity
s1_img_input_resampled = GeoUtils.resampledGeoTiff(s1_img_input, downSampleFactor)
s2_img_input_resampled = GeoUtils.resampledGeoTiff(s2_img_input, downSampleFactor)
s1_img_input_reprojected = GeoUtils.change_CRS_raster(s1_img_input_resampled, crs)
s2_img_input_reprojected = GeoUtils.change_CRS_raster(s2_img_input_resampled, crs)

## Use SAM for segmenting the plots

In [ ]:
# Segment the plots using Segment Anything Model
sam = SamGeo(checkpoint=sam_model_path, model_type='vit_h', sam_kwargs=None)
SegmentPlots.generateSegmentationMask(s1_img_input_resampled, sam)
SegmentPlots.generateSegmentationMask(s2_img_input_resampled, sam)

s1_input_raster = os.path.join(s1_FolderPath, s1_FileName + '-resampled-annotation.tif')
s1_output_vector = os.path.join(s1_FolderPath, s1_FileName + '-resampled-vector.shp')
s2_input_raster = os.path.join(s2_FolderPath, s2_FileName + '-resampled-annotation.tif')
s2_output_vector = os.path.join(s2_FolderPath, s2_FileName + '-resampled-vector.shp')
## Convert the rasterized segmentation results from SAM to vector format
SegmentPlots.raster_to_polygon(s1_input_raster, s1_output_vector)
SegmentPlots.raster_to_polygon(s2_input_raster, s2_output_vector)

## Remove outliers based on area of polygons and generate centroids on selected plots 

In [ ]:
s1_vector_input_Path = os.path.join(s1_FolderPath,s1_FileName+'-resampled-vector.shp')
s1_selected_polygons = SegmentPlots.selectPolygons(s1_vector_input_Path)
s2_vector_input_Path = os.path.join(s2_FolderPath,s2_FileName+'-resampled-vector.shp')
s2_selected_polygons = SegmentPlots.selectPolygons(s2_vector_input_Path)
s1_centroids = SegmentPlots.generateCentroids(s1_selected_polygons, s1_vector_input_Path)
s2_centroids = SegmentPlots.generateCentroids(s2_selected_polygons, s2_vector_input_Path)

## Generate feature descriptor for Reference Image

In [ ]:
s1_centroids_transformed_path = os.path.join(s1_FolderPath,s1_FileName+'-centroids-transfromed.shp')
# Identify and drop rows with duplicate values in the 'coordinate' column
s1_centroids = s1_centroids[~s1_centroids.duplicated(subset='geometry', keep='first')]
# Transform coordinates into EPSG: 4326
s1_centroids_transformed = GeoUtils.transformCRS(s1_centroids, crs)
# Sort the centroids based on latitude
s1_centroids_transformed = GeoUtils.sortGeoDataframe(s1_centroids_transformed)
# Assign id to each point
s1_centroids_transformed['id'] = range(1, len(s1_centroids_transformed) + 1)
# Calculate distance threshold
dist_dict = GeoFeatures.getDistanceDictionary(s1_centroids_transformed)
distanceThresholdNeighbors = GeoFeatures.getDistThresholdNeighbors(dist_dict, min_Neighbors, neighbor_percentile)
s1_neighbor_dist = GeoFeatures.get_neighbor_distances(s1_centroids_transformed, distanceThresholdNeighbors)
# Calculate feature vector for each feature
s1_featureDescriptor = GeoFeatures.get_feature_descriptor(s1_centroids_transformed,s1_neighbor_dist)
# Append the feature descriptor to the geopandas dataframe
s1_centroids_transformed = GeoFeatures.appendFeatureDescriptor(s1_centroids_transformed, s1_featureDescriptor) 
# Save the transformed centroid file
s1_centroids_transformed.to_file(s1_centroids_transformed_path, driver='ESRI Shapefile')

## Generate feature descriptor for Sensed Image

In [ ]:
s2_centroids_transformed_path = os.path.join(s2_FolderPath,s2_FileName+'-centroids-transfromed.shp')
# Identify and drop rows with duplicate values in the 'coordinate' column
s2_centroids = s2_centroids[~s2_centroids.duplicated(subset='geometry', keep='first')]
# Transform coordinates into EPSG: 4326
s2_centroids_transformed = GeoUtils.transformCRS(s2_centroids, crs)
# Sort the centroids based on latitude
s2_centroids_transformed = GeoUtils.sortGeoDataframe(s2_centroids_transformed)
# Assign id to each point
s2_centroids_transformed['id'] = range(1, len(s2_centroids_transformed) + 1)
s2_neighbor_dist = GeoFeatures.get_neighbor_distances(s2_centroids_transformed, distanceThresholdNeighbors)
# Calculate feature vector for each feature
s2_featureDescriptor = GeoFeatures.get_feature_descriptor(s2_centroids_transformed,s2_neighbor_dist)
# Append the feature descriptor to the geopandas dataframe
# Append the feature descriptor to the geopandas dataframe
s2_centroids_transformed = GeoFeatures.appendFeatureDescriptor(s2_centroids_transformed, s2_featureDescriptor) 
# Save the transformed centroid file
s2_centroids_transformed.to_file(s2_centroids_transformed_path, driver='ESRI Shapefile')

## Match centroids (Initial matching phase)

In [ ]:
#Calculate matches
matched_features = GeoFeatures.matchFeatures(s1_centroids_transformed, s2_centroids_transformed)
# Remove entries with empty list
matched_features = {key: value for key, value in matched_features.items() if value not in (None, [])}

matched_features_path = os.path.join(folderName,'Results','matchedFeatures-'+s1_FileName+'-'+s2_FileName+'.json')
# Save the dictionary to a JSON file
with open(matched_features_path, 'w') as json_file:
    json.dump(matched_features, json_file)
    
# with open(matched_features_path, 'r') as json_file:
#     matched_features = json.load(json_file)

## Save initial matches as line vectors

In [ ]:
matched_lines_path = os.path.join(folderName,'Results','matchedLines-'+s1_FileName+'-'+s2_FileName+'.shp')
matched_List = GeoFeatures.generateSrcToDstPointsList(s1_centroids_transformed, s2_centroids_transformed, matched_features)
GeoFeatures.drawLinesMatchedPoints(matched_List, matched_lines_path, crs);

## Remove outliers from initial match and populate additional matches

In [ ]:
matched_lines_inliers_path = os.path.join(folderName,'Results','matchedLinesInliers-'+s1_FileName+'-'+s2_FileName+'.shp')
add_matched_lines_inliers_path = os.path.join(folderName,'Results','addMatchedLinesInliers-'+s1_FileName+'-'+s2_FileName+'.shp')
(distList, brgList) = GeoFeatures.getDistBearingList(matched_List)
thresholds = GeoFeatures.getDistBearingThresholds(distList, brgList, bins_dist, bins_brg)
matched_features_inliers = GeoFeatures.removeOutlierPoints(s1_centroids_transformed, s2_centroids_transformed, matched_features, thresholds)
matched_inliers_List = GeoFeatures.generateSrcToDstPointsList(s1_centroids_transformed, s2_centroids_transformed, matched_features_inliers)
#open a fiona object
# lineShp = fiona.open(matched_lines_inliers_path, mode='w', driver='ESRI Shapefile',schema = schema, crs = crs)
GeoFeatures.drawLinesMatchedPoints(matched_inliers_List, matched_lines_inliers_path, crs)

add_match_thresholds = GeoFeatures.get_add_matches_theshold(matched_inliers_List)
add_matched_inliers_dict = GeoFeatures.add_feature_matches(s1_centroids_transformed, s2_centroids_transformed, add_match_thresholds)
add_matched_inliers_List = GeoFeatures.generateSrcToDstPointsList(s1_centroids_transformed, s2_centroids_transformed, add_matched_inliers_dict)
GeoFeatures.drawLinesMatchedPoints(add_matched_inliers_List, add_matched_lines_inliers_path, crs);

## Save final matched pairs

In [ ]:
add_matched_features_path = os.path.join(folderName,'Results','addMatchedLinesInliers-'+s1_FileName+'-'+s2_FileName+'.json')
# # Save the dictionary to a JSON file
with open(add_matched_features_path, 'w') as json_file:
    json.dump(add_matched_inliers_dict, json_file)
    
# with open(add_matched_features_path, 'r') as json_file:
#     add_matched_inliers_dict = json.load(json_file)

## Save final matched pairs as point files that can be directly used in QGIS for georeferencing

In [ ]:
GeoUtils.write_GCP_file(gcp_template_file_path, gcp_out_file, add_matched_inliers_List)